# Teste para fazer o target de acordo com KMeans

In [1]:
import csv
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import KMeans

In [2]:
df_original = pd.read_csv("/content/drive/MyDrive/Projetos_ion/ION_Smart_Locator/datasets/CNPJ_metadados/dados limpos/dataset_limpo_numeros.csv")
df_original.head()

,cnpj,cnae,capital_social,porte,Seção,Divisão,Grupo,Classe,idade_empresa,cnae_sec_1,...,Seção5,Divisão5,Grupo5,Classe5,target,target_Seção,target_Divisão,target_Grupo,target_Classe,target_cnae
0,35.464.865/0001-93,8230001,300.0,1,14,82,823,82300,1179,8230001,...,14,82,823,82300,0,-1,-1,-1,-1,-1
1,35.464.962/0001-86,4399103,5000.0,1,6,43,439,43991,1179,4321500,...,6,43,439,43991,0,-1,-1,-1,-1,0
2,35.465.399/0001-60,4520002,5000.0,1,7,45,452,45200,1179,4520005,...,7,45,452,45200,0,-1,-1,-1,-1,-1
3,35.465.446/0001-76,9602501,1000.0,1,19,96,960,96025,1179,9602501,...,19,96,960,96025,0,-1,-1,-1,-1,-1
4,35.465.492/0001-75,7739002,2000.0,1,14,77,773,77390,1179,7739002,...,14,77,773,77390,0,-1,-1,0,0,0


In [24]:
df_original.target.value_counts()

0    5795807
1        285
Name: target, dtype: int64

In [18]:
df_1_target = df_original.loc[(df_original['target'] == 1)]
df_1_target.reset_index(inplace=True)
df_1_target.drop(columns=['index'], inplace=True)
df_1_target11 = df_1_target.loc[:142,:]
df_1_target12 = df_1_target.loc[143:,:]

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [41]:
df_1_target.shape

(285, 40)

In [22]:
df_0_target = df_original.loc[(df_original['target'] == 0)]
df_0_target.shape

(5795807, 40)

In [23]:
df_0_target.reset_index(inplace=True)
df_0_target.drop(columns=['index'], inplace=True)
df_0_target11 = df_1_target.loc[:2897903,:]
df_0_target12 = df_1_target.loc[2897904:,:]

/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [25]:
df_1_target11.to_csv('1_new_dataset_target.csv', encoding='utf-8', index=False)

In [26]:
n_slice = int(len(df_0_target11) / len(df_1_target11))
slice_list = range(n_slice)

In [27]:
len_target_end_limit = 0

In [34]:
len_target_end_limit = 0
print("GERANDO CSVs INTERMEDIARIOS")
for idx in tqdm(slice_list):
    if idx != slice_list[-1]:
        len_target_start_limit = len_target_end_limit
        len_target_end_limit = len(df_1_target) * (idx + 1)
        df_0_target = df_0_target_original.iloc[len_target_start_limit:len_target_end_limit]
    else:
        df_0_target = df_0_target_original.iloc[len_target_end_limit:]
        df_1_target = df_1_target.iloc[:len(df_0_target)]
    df = pd.concat([df_1_target, df_0_target])
    data = df.copy()
    df.drop(['cnpj', 'target', 'target_Seção', 'target_Divisão', 'target_Grupo', 'target_Classe', 'target_cnae'],
            axis=1, inplace=True)
    df.columns = ['cnae', 'capital_social', 'porte', 'secao', 'divisao', 'grupo',
                  'classe', 'idade_empresa', 'cnae_sec_1', 'cnae_sec_2', 'cnae_sec_3',
                  'cnae_sec_4', 'cnae_sec_5', 'secao1', 'divisao1', 'grupo1', 'classe1',
                  'secao2', 'divisao2', 'grupo2', 'classe2', 'secao3', 'divisao3',
                  'grupo3', 'classe3', 'secao4', 'divisao4', 'grupo4', 'classe4',
                  'secao5', 'divisao5', 'grupo5', 'classe5']
    dataset = df.to_numpy()
    kmeans = KMeans(n_clusters=20, init='k-means++', n_init=10, max_iter=500)
    pred_y = kmeans.fit_predict(dataset)
    from statistics import mode

    mode = mode(pred_y)
    pred_y_2 = []
    for value in pred_y:
        if value == mode:
            pred_y_2.append(0)
        else:
            pred_y_2.append(1)
    pred_y = pred_y_2
    df['kmeans'] = pred_y
    df['tag'] = data['target']
    df_class_0 = df.loc[(df['tag'] == 1) & (df['kmeans'] == 0)]
    df_class_1 = df.loc[(df['tag'] == 1) & (df['kmeans'] == 1)]

    class_zero = len(df_class_0)
    class_one = len(df_class_1)

    perc_zero = round(len(df_class_0) / (len(df_class_0) + len(df_class_1)), 4)
    perc_one = round(len(df_class_1) / (len(df_class_0) + len(df_class_1)), 4)

    from random import *

    list_tag = df['tag'].values.tolist()
    list_kmeans = df['kmeans'].values.tolist()
    del df['tag']
    del df['kmeans']
    list_target = []
    for i in range(len(df)):
        value_tag = list_tag[i]
        if value_tag == 1:
            list_target.append(1)
            continue
        value_kmeans = list_kmeans[i]
        perc = randint(1, 100) / 100
        if value_kmeans == 0:
            if perc < perc_zero:
                list_target.append(1)
            else:
                list_target.append(0)
        else:
            if perc < perc_one:
                list_target.append(1)
            else:
                list_target.append(0)

    df['target'] = list_target
    df['cnpj'] = data['cnpj']
    df = df[['cnpj', 'cnae', 'capital_social', 'porte', 'secao', 'divisao', 'grupo',
             'classe', 'idade_empresa', 'cnae_sec_1', 'cnae_sec_2', 'cnae_sec_3',
             'cnae_sec_4', 'cnae_sec_5', 'secao1', 'divisao1', 'grupo1', 'classe1',
             'secao2', 'divisao2', 'grupo2', 'classe2', 'secao3', 'divisao3',
             'grupo3', 'classe3', 'secao4', 'divisao4', 'grupo4', 'classe4',
             'secao5', 'divisao5', 'grupo5', 'classe5', 'target']]
    df = df.iloc[len(df_1_target):]
    df.to_csv('csv/new_dataset_' + str(idx) + '.csv', encoding='utf-8', index=False)

df_write = pd.read_csv('csv/1_new_dataset_target.csv')
df_write.drop(['target_Seção', 'target_Divisão', 'target_Grupo', 'target_Classe', 'target_cnae'],
        axis=1, inplace=True)
df_write.columns = ['cnpj', 'cnae', 'capital_social', 'porte', 'secao', 'divisao', 'grupo',
             'classe', 'idade_empresa', 'cnae_sec_1', 'cnae_sec_2', 'cnae_sec_3',
             'cnae_sec_4', 'cnae_sec_5', 'secao1', 'divisao1', 'grupo1', 'classe1',
             'secao2', 'divisao2', 'grupo2', 'classe2', 'secao3', 'divisao3',
             'grupo3', 'classe3', 'secao4', 'divisao4', 'grupo4', 'classe4',
             'secao5', 'divisao5', 'grupo5', 'classe5', 'target']
print("GRAVANDO CSV FINAL")
df_write.to_csv('new_dataset.csv', encoding='utf-8', index=False)

for idx in tqdm(slice_list):
    with open('csv/new_dataset_' + str(idx) + '.csv', mode='r') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                with open('new_dataset.csv', 'a') as fd:
                    writer = csv.writer(fd)
                    writer.writerow(row)
                line_count += 1

GERANDO CSVs INTERMEDIARIOS


100%|██████████| 1/1 [00:00<00:00,  2.44it/s]


In [38]:
df=pd.read_csv('/content/new_dataset.csv')
df.head()

,cnpj,cnae,capital_social,porte,secao,divisao,grupo,classe,idade_empresa,cnae_sec_1,...,classe3,secao4,divisao4,grupo4,classe4,secao5,divisao5,grupo5,classe5,target
0,40.223.835/0001-25,4771701,90000.0,1,7,47,477,47717,762,4721104,...,47296,7,47,476,47610,7,47,477,47717,1
1,05.010.690/0001-83,4771701,0.0,1,7,47,477,47717,7600,4771701,...,47717,7,47,477,47717,7,47,477,47717,1
2,05.166.298/0001-28,4771701,40000.0,1,7,47,477,47717,7509,4772500,...,47717,7,47,477,47717,7,47,477,47717,1
3,02.633.061/0001-11,4771701,45000.0,3,7,47,477,47717,8964,4761003,...,47211,7,47,472,47296,7,46,461,46192,1
4,18.120.089/0001-95,4771701,5000.0,1,7,47,477,47717,14448,4771701,...,47717,7,47,477,47717,7,47,477,47717,1


In [39]:
df.shape

(428, 35)

In [40]:
df.target.value_counts()

1    428
Name: target, dtype: int64